# Please Do Upvote If You Like The Notebook.

## I Got ROC_AUC_SCORE : 0.858765 On Analystics Vidhya Leaderboard. Rank : 48.

### This Notebook Contains : 
    1. Deep Exploratory Data Analysis
    2. Outlier Analysis
    3. Corelation Analysis
    4. Data Preprocessing
    5. Model Building
    6. Hyperparameter Tuning
    7. Submission With Short Ensembling

# Importing Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualizatin
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
sns.set(style="ticks", context="talk")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import optuna
from optuna.samplers import TPESampler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading Data

In [ ]:
train = pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
test = pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')
sample_sub = pd.read_csv('../input/health-insurance-cross-sell-prediction/sample_submission.csv')

In [ ]:
print('Shape of train: {}'.format(train.shape))
print('Shape of test: {}'.format(test.shape))

In [ ]:
train.head()

# Data Dictionary

id:	Unique ID for the customer

Gender:	Gender of the customer

Age: Age of the customer

Driving_License: 	0 : Customer does not have DL, 1 : Customer already has DL

Region_Code:	Unique code for the region of the customer

Previously_Insured: 	1 : Customer already has Vehicle Insurance, 0 : Customer doesn't have Vehicle Insurance

Vehicle_Age: 	Age of the Vehicle 

Vehicle_Damage: 
1 : Customer got his/her vehicle damaged in the past.
0 : Customer didn't get his/her vehicle damaged in the past.

Annual_Premium: The amount customer needs to pay as premium in the year

Policy_Sales_Channel:	Anonymised Code for the channel of outreaching to the customer ie. Different Agents, Over Mail, Over Phone, In Person, etc.

Vintage:	Number of Days, Customer has been associated with the company

Response:	1 :  Customer is interested, 0 : Customer is not interested

## Encoding Target Variable

In [ ]:
train['Response'] = le.fit_transform(train['Response'])

# Exploratory Data Analysis

### Checking Target Variable Bias

In [ ]:
train['Response'].value_counts().plot.pie(autopct = '%1.1f%%',colors=['Orange','Blue'], figsize = (7,7))

## Data is Imbalanced. Only 12.3% of customers are likely to buy insuarance.

## Missing Value Analysis

In [ ]:
train.isna().sum()/train.shape[0]*100

There are no missing values

## Analysing Each Variable's Relationship with Target Variablem

### 1. Gender

In [ ]:
sns.countplot(train['Gender'], hue = train['Response'],palette=['Orange','Purple'])

## 2. Age

In [ ]:
f,ax = plt.subplots(nrows=2,ncols=1,figsize=(30,10))
axx = ax.flatten()
#plt.figure(figsize=(30,10))
sns.distplot(train['Age'],ax=axx[0], color='Blue')
sns.boxplot(train['Age'],ax=axx[1],color='Orange')

### Analysing Response for different Age-Groups

In [ ]:
age_grp_20_to_30 = train[ train['Age'] <31]
age_grp_31_to_40 = train[ train['Age'].between(31,40)]
age_grp_41_to_50 = train[ train['Age'].between(41,50)]
age_grp_50_to_60 = train[ train['Age'].between(51,60)]
age_grp_old = train[ train['Age'] >60]

age_grp = [age_grp_20_to_30,age_grp_31_to_40,age_grp_41_to_50,age_grp_50_to_60,age_grp_old]
age_grp_name = ['age_grp_20_to_30','age_grp_31_to_40','age_grp_41_to_50','age_grp_50_to_60','age_grp_old']
age_grp_dict = dict(zip(age_grp_name, age_grp))

In [ ]:
f,ax = plt.subplots(nrows=2, ncols=3, figsize = (20,10))
axx = ax.flatten()
for pos,tup in enumerate(age_grp_dict.items()):
    axx[pos].set_title(tup[0])
    data = tup[1]
    data['Response'].value_counts().plot.pie(autopct='%1.1f%%', ax = axx[pos],colors=['Red','Blue'])

### Analysing Response with both 'Age-Groups' and 'Gender'

In [ ]:
f,ax = plt.subplots(nrows=2, ncols=3, figsize = (20,10))
axx = ax.flatten()
plt.title('Response Percentage of Different Age Groups with Genders',fontsize=40,x=-0.5,y=2.5)
for pos,tup in enumerate(age_grp_dict.items()):
    axx[pos].set_title(tup[0])
    temp = tup[1]
    temp.groupby('Gender')['Response'].value_counts().plot.pie(autopct='%1.1f%%', ax = axx[pos],colors=['Orange','Purple'])

In [ ]:
sns.catplot(x = 'Gender', y="Age",hue = 'Response', data=train)

## Insights:

1. Customers of age between 30 to 60 are more likely to buy insurance.
2. Customes of age between 20 to 30 are less likely to buy insurance.
3. In almost every age group, 'Male's are more likely to buy insurance.
4. Females under age 30 are very less likely ho buy insurance

---
## 3. Driving_License

In [ ]:
train['Driving_License'].value_counts().plot.pie(autopct='%1.1f%%',colors = ['Blue','Red'])

In [ ]:
f,ax = plt.subplots(nrows=1,ncols=2,figsize = (20,5))
axx = ax.flatten()
#plt.title('Driving_License wise Response',fontsize=40,x=-0.5,y=2)
axx[0].set_title('Driving_Licence = 1')
axx[1].set_title('Driving_Licence = 0')
train[ train['Driving_License'] == 1]['Response'].value_counts().plot.pie(autopct='%1.1f%%',colors = ['Blue','Red'],ax=axx[0])
train[ train['Driving_License'] == 0]['Response'].value_counts().plot.pie(autopct='%1.1f%%',colors = ['Blue','Red'],ax=axx[1])

### Insights:

1. Very few customers don't have Driving License.
2. Customers with Driving License have higher chance of buying Insurancem

---
## 4. Region_Code

In [ ]:
plt.figure(figsize = (40,10))
plt.title('Region Wise Response Count',fontsize=50)
sns.countplot(train['Region_Code'], hue = train['Response'],palette=['Red','Blue'])

### Visualizing Percentage of Response : 1 in all Region_Codes

In [ ]:
u_region = train['Region_Code'].unique()
region_perc = {}
for i in u_region:
    total_region = train[ train['Region_Code'] == i].shape[0]
    buy_region = train[ (train['Region_Code'] == i) & train['Response'] == 1].shape[0]
    region_perc[i] = (buy_region/total_region)*100

region_perc = sorted(region_perc.items(), key=lambda x: x[1], reverse=True)
region_perc = list(zip(*region_perc))

region = np.array(region_perc[0])
region_perc = np.array(region_perc[1])
region = pd.DataFrame(region)
region_perc = pd.DataFrame(region_perc)

region_res_perc = pd.concat((region,region_perc), axis=1)
region_res_perc.columns = ['Region_Code', 'Buy_Percentage']

In [ ]:
plt.figure(figsize=(40,10))
plt.title('Region Wise Buying Percentage',fontsize=50)
ax = sns.barplot(x = region_res_perc['Region_Code'], y = region_res_perc['Buy_Percentage'])

### Insights:

1. We have most of the customers from Region_Code : 28.
2. Region_Codes: [4,19,23,24,,28,38,51] have higher percentage of buying insurance.
3. Region_Codes: 25 and 44 have lower percentage of buying insurance.

---
## 5. Previously_Insuaranced

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(train['Previously_Insured'],hue=train['Response'],palette=['Brown','Purple'])

### Insights:

1. Customers who Previously_Insured are very likely to buy Insurnce now.
2. Customers who didn't Previously_Insured have good chance of buying Insurnce.

---
## 6. Vehicle_Age

In [ ]:
plt.figure(figsize=(7,7))
train['Vehicle_Age'].value_counts().plot.pie(autopct='%1.1f%%', colors = ['r', 'b', 'g'])

In [ ]:
plt.figure(figsize = (30,10))
sns.countplot(train['Vehicle_Age'], hue = train['Response'])

In [ ]:
ls = train['Vehicle_Age'].unique()

In [ ]:
f,ax = plt.subplots(nrows=1, ncols=3,figsize = (30,10))
axx = ax.flatten()
for pos,val in enumerate(ls):
    axx[pos].set_title(str(val))
    train[ train['Vehicle_Age'] == val]['Response'].value_counts().plot.pie(autopct = '%1.1f%%',ax = axx[pos], colors=['Purple', 'Orange'])

### Insights :

1. We have half of our customers with Vehicle_Age `1-2 years`.
2. We have very few customers (4.2%) with Vehicle_Age `>2 years.
3. Customers with Vehicle_Age `>2years` have better chance (29.4%) of buying Insurance.
4. Customers with with Vehicle_Age `<1 years` have very less chance of buying Insurance.

---
## 7. Vehicle_Damage

In [ ]:
sns.countplot(train['Vehicle_Damage'], hue = train['Response'])

### Insights:

1. We have almost same number of customes with damaged and non_damaged vehicle.
2. Customers with Vehicle_Damage are likely to buy insurance.
3. Customers with non damaged vehicle have least chance (less than 1%) of buying insurance.

---
## 8. Annual_Premium

In [ ]:
f,ax = plt.subplots(nrows=2,ncols=1,figsize=(30,20))
axx = ax.flatten()
#plt.figure(figsize=(30,10))
sns.distplot(train['Annual_Premium'],ax=axx[0], color='Blue')
sns.boxplot(train['Annual_Premium'],ax=axx[1],color='Orange')

In [ ]:
plt.figure(figsize=(40,10))
sns.distplot(train[ train['Annual_Premium'] < 100000]['Annual_Premium'])#.plot.hist(bins = 500, frequency=(0,10000))

In [ ]:
start = 0
step = 10000
ls = []
for _ in range(10):
    ls.append((start,step))
    start = step
    step+=10000

for tup in ls:
    count = train[ train['Annual_Premium'].between(tup[0],tup[1])].shape[0]
    percentage = train[ (train['Annual_Premium'].between(tup[0], tup[1])) & (train['Response'] == 1)].shape[0]/train[ train['Annual_Premium'].between(tup[0], tup[1])].shape[0]*100
    print('Number of Customers with Annual_Premium Between {} : {} and Insurance Buy Percentage:{}'.format(tup,count,percentage))

### Insights:

1. 'Annual Premium' data is highlt left skewed.
2. Most of the customers have "Annual_Premium' in range (0, 10000) and (20000 to 50000)
3. In every 'Annual Premium' range, the insurance buy percentage is almost same.

---
## 9. Plolicy_Sales_Channel

In [ ]:
plt.figure(figsize=(40,10))
train['Policy_Sales_Channel'].value_counts().plot.bar()

### Insights:

1. Policy_Sales_Channel no. 152 have higest number of customers.
2. Policy_Sales_Channel no. [152,26,124,160,156,122,157,154,151,163] have most of the customers.

---
## 10. Vintage

In [ ]:
f,ax = plt.subplots(nrows=2,ncols=1,figsize=(30,20))
axx = ax.flatten()
sns.distplot(train['Vintage'],ax=axx[0], color='Blue')
sns.boxplot(train['Vintage'],ax=axx[1],color='Orange')

### Insights:

1. Every 'Vintage' value have almost same number of customers.

---
# Everything at Once.

### All Insights From Variable Analysis



1. Customers of age between 30 to 60 are more likely to buy insurance.
2. Customes of age between 20 to 30 are less likely to buy insurance.
3. In almost every age group, 'Male's are more likely to buy insurance.
4. Females under age 30 are very less likely ho buy insurance.
5. Very few customers don't have Driving License.
6. Customers with Driving License have higher chance of buying Insurance.
7. We have most of the customers from Region_Code : 28.
8. Region_Codes: [4,19,23,24,,28,38,51] have higher percentage of buying insurance.
9. Region_Codes: 25 and 44 have lower percentage of buying insurance.
10. Customers who Previously_Insured are very likely to buy Insurnce now.
11. Customers who didn't Previously_Insured have good chance of buying Insurnce.
12. We have half of our customers with Vehicle_Age 1-2 years.
13. We have very few customers (4.2%) with Vehicle_Age >2 years.
14. Customers with Vehicle_Age >2years have better chance (29.4%) of buying Insurance.
15. Customers with with Vehicle_Age <1 years have very less chance of buying Insurance.
16. We have almost same number of customes with damaged and non_damaged vehicle.
17. Customers with Vehicle_Damage are likely to buy insurance.
18. Customers with non damaged vehicle have least chance (less than 1%) of buying insurance.
19. 'Annual Premium' data is highlt left skewed.
20. Most of the customers have "Annual_Premium' in range (0, 10000) and (20000 to 50000)
21. In every 'Annual Premium' range, the insurance buy percentage is almost same.
22. Policy_Sales_Channel no. 152 have higest number of customers.
23. Policy_Sales_Channel no. [152,26,124,160,156,122,157,154,151,163] have most of the customers.
24. Every 'Vintage' value have almost same number of customers.



---
# Merging Train and Test Data

In [ ]:
train['is_train'] = 1
test['is_train'] = 0
test['Response'] = None

data = pd.concat((train,test))
data.set_index('id',inplace=True)
data.shape

---
# Outlier Analysis

## 1. Age

In [ ]:
sns.boxplot('Age', data=data, orient='v', color='Red')

## 2. Annual_Premium

In [ ]:
sns.boxplot('Annual_Premium', data=data,orient='v', color='red')

In [ ]:
f,ax = plt.subplots(nrows=1,ncols=2,figsize = (40,10))
axx = ax.flatten()
sns.kdeplot(data['Annual_Premium'], legend=False,ax = axx[0])
sns.kdeplot(np.log(data['Annual_Premium']), legend=False,ax = axx[1]) # after using log transformation

### we will do log transformation on 'Annual_Premium'm

---
# Corelation Analysis

### Label Encoding for co-relation checking

In [ ]:
corr_check = data.copy()

col_ls = ['Gender', 'Vehicle_Age', 'Vehicle_Damage']

for col in col_ls:
    corr_check[col] = le.fit_transform(corr_check[col])

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(corr_check.corr(), annot=True, square=True,annot_kws={'size': 10})

## Insights: 

1. 'Previously_Insured'  and 'Vehicle_Damage' are highly positively corelated.
2. 'Age' and 'Policy_Sales_Channel' are negatively corelated.
3. 'Age' and 'Vehicle_Age' are negatively corelated.

---
# Data Preprocessing

In [ ]:
train['Vehicle_Age']=train['Vehicle_Age'].replace({'< 1 Year':0,'1-2 Year':1,'> 2 Years':2})
train['Gender']=train['Gender'].replace({'Male':1,'Female':0})
train['Vehicle_Damage']=train['Vehicle_Damage'].replace({'Yes':1,'No':0})

test['Vehicle_Age']=test['Vehicle_Age'].replace({'< 1 Year':0,'1-2 Year':1,'> 2 Years':2})
test['Gender']=test['Gender'].replace({'Male':1,'Female':0})
test['Vehicle_Damage']=test['Vehicle_Damage'].replace({'Yes':1,'No':0})

In [ ]:
# Changing Datatype
train['Region_Code']=train['Region_Code'].astype(int)
test['Region_Code']=test['Region_Code'].astype(int)
train['Policy_Sales_Channel']=train['Policy_Sales_Channel'].astype(int)
test['Policy_Sales_Channel']=test['Policy_Sales_Channel'].astype(int)

---
# Model Building

In [ ]:
features=['Gender', 'Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']

cat_col=['Gender','Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage','Policy_Sales_Channel']

---
## Train Test Split

In [ ]:
X=train[features]
y=train['Response']

In [ ]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=150303,stratify=y,shuffle=True)

## 1. CatBoost Model

In [ ]:
catb = CatBoostClassifier()
catb= catb.fit(X_train, y_train,cat_features=cat_col,eval_set=(X_test, y_test),early_stopping_rounds=30,verbose=100)
y_pred = catb.predict(X_test)
proba = catb.predict_proba(X_test)[:, 1]
print('CatBoost Base Accuracy : {}'.format(accuracy_score(y_test,y_pred)))
print('CatBoost Base ROC_AUC_SCORE: {}'.format(roc_auc_score(y_test,proba)))

## 2. LGBM Model

In [ ]:
model = LGBMClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
proba = model.predict_proba(X_test)[:,1]

In [ ]:
print('LGBM Base Accuracy : {}'.format(accuracy_score(y_test,y_pred)))
print('LGBM Base ROC_AUC_SCORE: {}'.format(roc_auc_score(y_test,proba)))

## Hyperparameter Tuning LGBM

In [ ]:
def create_model(trial):
    max_depth = trial.suggest_int("max_depth", 2, 30)
    n_estimators = trial.suggest_int("n_estimators", 1, 500)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0000001, 1)
    num_leaves = trial.suggest_int("num_leaves", 2, 5000)
    min_child_samples = trial.suggest_int('min_child_samples', 3, 200)
    reg_alpha = trial.suggest_int("reg_alpha", 1, 10)
    reg_lambda = trial.suggest_int("reg_lambda", 1, 10)
    model = LGBMClassifier(
        learning_rate=learning_rate, 
        n_estimators=n_estimators, 
        max_depth=max_depth,
        num_leaves=num_leaves, 
        min_child_samples=min_child_samples,
        random_state=0
    )
    return model

sampler = TPESampler(seed=0)
def objective(trial):
    model = create_model(trial)
    model.fit(X_train, y_train)
    proba = model.predict_proba(X_test)[:,1]
    score = roc_auc_score(y_test,proba)
    return score

study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=50)

lgb_params = study.best_params
lgb_params['random_state'] = 0
lgb = LGBMClassifier(**lgb_params)
lgb.fit(X_train, y_train)
proba = lgb.predict_proba(X_test)[:,1]
print('Optimized LightGBM roc_auc_score', roc_auc_score(y_test, proba))

In [ ]:
lgb

In [ ]:
LGBM = lgb
LGBM.fit(X, y)
y_pred = LGBM.predict(X_test)
proba = LGBM.predict_proba(X_test)[:,1]

In [ ]:
print('LGBM Tuned Accuracy : {}'.format(accuracy_score(y_test,y_pred)))
print('LGBM Tuned ROC_AUC_SCORE: {}'.format(roc_auc_score(y_test,proba)))

---
## Short Ensemble

### Combining Results of CatBoost and LGBM for final submission


In [ ]:
LGBM_proba = LGBM.predict_proba(test[features])[:, 1] # Class 1 probability of LGBM model
cat_proba = catb.predict_proba(test[features])[:, 1] # Class 1 probability of CatBoost model

#### Taking Weighted Average.

In [ ]:
submit_proba = ((LGBM_proba * 0.45) + (cat_proba * 0.55))/2

sample_sub['Response'] = submit_proba

# sample_sub.to_csv() --- > Add your path here

# Please Upvote if you like the notebook. Feel free to give suggestions about improving my work. Thank You.